# eval

In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression
import pickle
import yaml
from tqdm import tqdm
import jax.numpy as jnp
from src.utils import get_args_and_config, SUBJECTS
from src.data import get_data
from src.eval import corr
from src.fmri import plot_brain

100%|██████████| 6/6 [00:07<00:00,  1.19s/it]


In [2]:
args, config = get_args_and_config()
data = get_data(args)

  0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
def algonauts_model(subject, train_data, test_data):
    """algonauts_model function"""
    train_data_img = np.vstack([x[0] for x in train_data])
    train_data_lh = np.vstack([x[1] for x in train_data])
    train_data_rh = np.vstack([x[2] for x in train_data])
    train_data = (train_data_img, train_data_lh, train_data_rh)

    lh_model = LinearRegression().fit(train_data_img, train_data_lh)
    rh_model = LinearRegression().fit(train_data_img, train_data_rh)

    # save model
    # pickle.dump(lh_model, open(f'./models/{subject}_lh_algonauts_model.pkl', 'wb'))
    # pickle.dump(rh_model, open(f'./models/{subject}_rh_algonauts_model.pkl', 'wb'))

    # test model
    test_data_img, test_data_lh, test_data_rh, _ = test_data
    
    test_lh_pred = lh_model.predict(test_data_img)
    test_rh_pred = rh_model.predict(test_data_img)
    test_lh_corr = corr(test_lh_pred, test_data_lh)
    test_rh_corr = corr(test_rh_pred, test_data_rh)
    return test_lh_corr, test_rh_corr


lh_corrs, rh_corrs = [], []
for subject in tqdm(data.keys()):
    test_lh_corr, test_rh_corr = algonauts_model(subject, data[subject][0], data[subject][1])
    break

plot_brain(test_lh_corr, 'subject', 'lh')

100%|██████████| 6/6 [00:47<00:00,  7.84s/it]


(0.20836793, 0.20575279)

In [10]:
data = {}
for idx, subj in enumerate(SUBJECTS):
    data[subj] = {'lh': lh_corrs[idx], 'rh': rh_corrs[idx]}

In [12]:
with open('./corrs.csv', 'w') as f:
    f.write('subject,lh,rh\n')
    for subj in data.keys():
        f.write(f'{subj},{data[subj]["lh"]},{data[subj]["rh"]}\n')

In [13]:
!ls

Dockerfile     corrs.csv  main.py      pyproject.toml	 src
README.md      data	  models       report		 streamlit_app.py
config	       jobs	  notebooks    requirements.txt  wandb
container.sif  logs	  poetry.lock  setup.py
